<a href="https://colab.research.google.com/github/skssushil/Web-mining/blob/master/1_3Getting_data_from_google_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import nltk
import re
import math
import numpy as np
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")

In [3]:
from google.colab import files
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
# to search
query =["tutorial on datastructure"]
url=[]
for j in search(query[0], tld="com", num=5, stop=10, pause=2):
  url.append(j)
print(url)

['https://www.tutorialspoint.com/data_structures_algorithms/index.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/data_structures_basics.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/algorithms_basics.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/graph_data_structure.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/data_structure_overview.htm', 'https://www.geeksforgeeks.org/data-structures/', 'https://www.javatpoint.com/data-structure-tutorial', 'https://www.javatpoint.com/data-structure-interview-questions', 'https://www.javatpoint.com/data-structure-introduction', 'https://www.javatpoint.com/singly-linked-list']


In [4]:
doc=[]
for k in url:
  req = urllib.request.Request(k, headers={'User-Agent': 'Mozilla/5.0'})
  data = urllib.request.urlopen(req).read()
  html= data.decode()
  soup = BeautifulSoup(html,features="lxml")
  for script in soup(["script", "style"]):
    script.extract()
  # get text
  text = soup.get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split(" "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  doc.append(text)

In [5]:
N=len(doc)

In [6]:
N

10

##Preprocessing

In [7]:
def convert_lower_case(data):
  return np.char.lower(data)

In [8]:
def remove_string_special_characters(data):
  # removes special characters with ' '
  stripped = re.sub('[^a-zA-z\s]', '', data)
  stripped = re.sub('_', '', stripped)
  # Change any white space to one space
  stripped = re.sub('\s+', ' ', stripped)
  # Remove start and end white spaces
  stripped = stripped.strip()
  return stripped.lower()

In [9]:
def stemming(data):
  stemmer= PorterStemmer()    
  tokens = word_tokenize(str(data))
  new_text = ""
  for w in tokens:
    new_text = new_text + " " + stemmer.stem(w)
  return new_text

In [10]:
def convert_numbers(data):
  tokens = word_tokenize(str(data))
  new_text = ""
  for w in tokens:
    try:
      w = num2words(int(w))
    except:
      a = 0
      new_text = new_text + " " + w
  new_text = np.char.replace(new_text, "-", " ")
  return new_text

In [11]:
# Stopword removal
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [12]:
def preprocess(data):
  Clean_data= remove_string_special_characters(data)  # removing special character
  Clean_data=stemming(Clean_data)
  Clean_data= convert_numbers(Clean_data)
  Clean_data=remove_stop_words(Clean_data)
  return Clean_data

# preprocessing of data completed

In [13]:
processed_text = []
for i in doc[:N]:
    processed_text.append(word_tokenize(str(preprocess(i))))

In [14]:
len(processed_text)

10

In [15]:
DF = {}
for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [16]:
total_vocab_size = len(DF)
print(total_vocab_size)

1508


In [17]:
total_vocab = [x for x in DF]

In [18]:
print(total_vocab[:20])

['data', 'structur', 'algorithm', 'tutori', 'tutorialspoint', 'home', 'job', 'tool', 'code', 'ground', 'current', 'affair', 'upsc', 'note', 'onlin', 'tutor', 'whiteboard', 'net', 'meet', 'tutorix']


In [19]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [20]:
doc = 0
tf_idf = {}
df_score={}
for i in range(N):
  tokens = processed_text[i]    
  counter = Counter(tokens)
  words_count = len(tokens)    
  for token in np.unique(tokens):
    df_score[doc,token]=counter[token]/words_count
    tf = counter[token]/words_count
    df = doc_freq(token)
    idf = np.log((N+1)/(df+1))        
    tf_idf[doc, token] = tf*idf
  doc += 1

In [21]:
idf=sorted(df_score.items(), key=lambda x: x[1], reverse=True)
tfidf=sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)

In [22]:
idf[:20]

[((1, 'dsa'), 0.09243697478991597),
 ((3, 'dsa'), 0.08870967741935484),
 ((1, 'data'), 0.08403361344537816),
 ((7, 'interview'), 0.07942079859587538),
 ((0, 'dsa'), 0.0763888888888889),
 ((2, 'algorithm'), 0.07053469852104664),
 ((4, 'dsa'), 0.06842923794712286),
 ((4, 'data'), 0.06687402799377916),
 ((8, 'data'), 0.06619593998234775),
 ((0, 'data'), 0.0625),
 ((6, 'program'), 0.06178707224334601),
 ((6, 'list'), 0.060836501901140684),
 ((6, 'link'), 0.058935361216730035),
 ((9, 'node'), 0.05256648113790971),
 ((2, 'dsa'), 0.05005688282138794),
 ((5, 'tree'), 0.048601558917927556),
 ((0, 'structur'), 0.043402777777777776),
 ((4, 'structur'), 0.041990668740279936),
 ((9, 'list'), 0.04143475572047001),
 ((8, 'structur'), 0.03971756398940865)]

In [23]:
tfidf[:20]

[((1, 'dsa'), 0.05602935999389471),
 ((3, 'dsa'), 0.05377011160704412),
 ((0, 'dsa'), 0.046302040550510216),
 ((4, 'dsa'), 0.04147741113078364),
 ((6, 'ds'), 0.030728851497086578),
 ((2, 'dsa'), 0.0303412688931671),
 ((9, 'ptr'), 0.026515979267964507),
 ((5, 'articl'), 0.025012351651366165),
 ((9, 'node'), 0.023759267481855202),
 ((9, 'enter'), 0.023193851595080615),
 ((1, 'builtin'), 0.021488421330736453),
 ((9, 'locat'), 0.020644916564866934),
 ((2, 'step'), 0.02063085243273972),
 ((3, 'vertic'), 0.01907558129913557),
 ((4, 'entiti'), 0.018185920462165394),
 ((6, 'node'), 0.01718574614992613),
 ((9, 'null'), 0.016070290465433037),
 ((3, 'edg'), 0.015896317749279642),
 ((3, 'vertex'), 0.015717132872543477),
 ((5, 'cs'), 0.01485108379299866)]